# The Impact of Litigations on GHG Emissions
* "litigation" from [Graham Institute](https://climate-laws.org/legislation_and_policies), takes the value of 1 when the first case related to environmental laws was solved in a given year, independently of the result
* "rule of law" from World Bank
* Sample excludes the US, and includes all countries for which there has been at least a case with a resolution: 23 countries, mostly HI and MI countries

##### Load R packages

In [1]:
options(conflicts.policy = list(warn = FALSE))
library("dplyr")
library("estimatr")
library("fixest")
library("ggplot2")
library("gmm")
library("lmtest")
library("plm")
library("readr")
library("rstatix")

Loading required package: sandwich

Loading required package: zoo



##### Read data

In [2]:
my_data <- read_csv("../data/litigclean_eps.csv", show_col_types = 'FALSE')

##### Define Country_ID as numerical

In [3]:
# egen id = group(country)
my_data <- my_data %>% group_by(country) %>% mutate(id = cur_group_id())

In [4]:
# xtset id year
plm::is.pbalanced(my_data$id, my_data$year)

[1] TRUE

##### Creation of log values

In [5]:
my_data$lnghg <- log(my_data$ghg)
my_data$lngdp <- log(my_data$gdp)
my_data$lngdp2 <- log(my_data$gdp)^2
my_data$lnpopulation <- log(my_data$population)
my_data$lnlaw <- log(my_data$ruleoflaw)

##### Definition of High-Income and Middle-Income countries

In [6]:
HI <- my_data$country %in% c('BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                             'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                             'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU')


In [7]:
MI <- my_data$country %in% c('BGR', 'CRI', 'FJI', 'IND', 'KEN', 
                             'PAK', 'NPL', 'PHL', 'UKR')

(2) TW-FE

In [8]:
# xtreg lnghg lngdp lngdp2 lnpop lnlaw litigation eps i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation + eps,
                  index = c("id", "year"), model="within", effect='twoways', 
                  data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation + eps, data = my_data, effect = "twoways", model = "within", 
    index = c("id", "year"))

Unbalanced Panel: n = 18, T = 19-20, N = 359

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.1856773 -0.0348761 -0.0014917  0.0337854  0.1519308 

Coefficients:
              Estimate Std. Error t-value  Pr(>|t|)    
lngdp         2.345554   0.409284  5.7309 2.331e-08 ***
lngdp2       -0.094936   0.023563 -4.0291 7.020e-05 ***
lnpopulation  1.244627   0.243557  5.1102 5.581e-07 ***
lnlaw        -0.119600   0.018605 -6.4282 4.758e-10 ***
litigation   -0.031540   0.015878 -1.9863   0.04786 *  
eps          -0.027488   0.020243 -1.3579   0.17546    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    3.3802
Residual Sum of Squares: 1.0695
R-Squa

In [9]:
# xtreg lnghg lngdp lngdp2 lnpop lnlaw litigation eps_* i.year,fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation + eps_mk + eps_nm + eps_ts,
                  index = c("id", "year"), model="within", effect='twoways', 
                  data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation + eps_mk + eps_nm + eps_ts, data = my_data, effect = "twoways", 
    model = "within", index = c("id", "year"))

Unbalanced Panel: n = 18, T = 19-20, N = 359

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.1824098 -0.0348066 -0.0019761  0.0342712  0.1553823 

Coefficients:
               Estimate Std. Error t-value  Pr(>|t|)    
lngdp         2.2480150  0.3838961  5.8558 1.196e-08 ***
lngdp2       -0.0898721  0.0229929 -3.9087 0.0001137 ***
lnpopulation  1.2895690  0.2298957  5.6094 4.458e-08 ***
lnlaw        -0.1177078  0.0210098 -5.6025 4.622e-08 ***
litigation   -0.0350416  0.0161779 -2.1660 0.0310617 *  
eps_mk       -0.0282713  0.0214096 -1.3205 0.1876308    
eps_nm       -0.0034705  0.0070854 -0.4898 0.6246115    
eps_ts       -0.0082549  0.0098566 -0.8375 0.4029444 

In [11]:
# xtreg lnghg lngdp lngdp2 lnpop litigation c.lnlaw##c.eps_* i.year,fe robust

my_data$lnlaw_eps_mk <- my_data$lnlaw * my_data$eps_mk
my_data$lnlaw_eps_nm <- my_data$lnlaw * my_data$eps_nm
my_data$lnlaw_eps_ts <- my_data$lnlaw * my_data$eps_ts

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation + eps_mk + eps_nm + eps_ts
                  + lnlaw_eps_mk + lnlaw_eps_nm + lnlaw_eps_ts,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation + eps_mk + eps_nm + eps_ts + lnlaw_eps_mk + lnlaw_eps_nm + 
    lnlaw_eps_ts, data = my_data, effect = "twoways", model = "within", 
    index = c("id", "year"))

Unbalanced Panel: n = 18, T = 19-20, N = 359

Residuals:
       Min.     1st Qu.      Median     3rd Qu.        Max. 
-0.17997882 -0.03514777 -0.00074481  0.03414351  0.14014233 

Coefficients:
              Estimate Std. Error t-value  Pr(>|t|)    
lngdp         2.588482   0.654524  3.9548 9.492e-05 ***
lngdp2       -0.105623   0.037091 -2.8477  0.004697 ** 
lnpopulation  1.120063   0.232115  4.8255 2.192e-06 ***
lnlaw        -0.366093   0.144774 -2.5287  0.011942 *  
litigation   -0.030877   0.015830 -1.9505  0.052019 .  
eps_mk       -0.338599   0.178957 -1.8921  0.059411 .  
eps_nm       -0.125692   0.068677 -1.8302  0.068177 .  
e

In [12]:
# xtreg lnghg lngdp lngdp2 lnpop lnlaw i.litigation##c.eps_* i.year,fe robust

my_data$litigation_eps_mk <- my_data$litigation * my_data$eps_mk
my_data$litigation_eps_nm <- my_data$litigation * my_data$eps_nm
my_data$litigation_eps_ts <- my_data$litigation * my_data$eps_ts

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation + eps_mk + eps_nm + eps_ts
                  + litigation_eps_mk + litigation_eps_nm + litigation_eps_ts,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation + eps_mk + eps_nm + eps_ts + litigation_eps_mk + 
    litigation_eps_nm + litigation_eps_ts, data = my_data, effect = "twoways", 
    model = "within", index = c("id", "year"))

Unbalanced Panel: n = 18, T = 19-20, N = 359

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.1882849 -0.0318157 -0.0012683  0.0321062  0.1663998 

Coefficients:
                    Estimate Std. Error t-value  Pr(>|t|)    
lngdp              2.2150624  0.3113362  7.1147 7.767e-12 ***
lngdp2            -0.0880871  0.0202922 -4.3409 1.921e-05 ***
lnpopulation       1.2414333  0.2191743  5.6641 3.365e-08 ***
lnlaw             -0.1070925  0.0223811 -4.7849 2.647e-06 ***
litigation         0.0761786  0.0909742  0.8374 0.4030307    
eps_mk            -0.0012342  0.0177631 -0.0695 0.9446491    
eps_nm    